In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 網址後方加上 MARKET:STOCK_ID 即為個股資訊. e.g, TPE:2330
G_FINANCE_URL = 'https://www.google.com/search?q='
G_FINANCE_HIS_URL = 'https://finance.google.com/finance/historical?q='


def get_web_page(url, stock_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                             'AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/66.0.3359.181 Safari/537.36'}
    resp = requests.get(url + stock_id, headers=headers)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

if __name__ == '__main__':
    page = get_web_page(G_FINANCE_URL, 'TPE:2330')
    if page:
        
        soup = BeautifulSoup(page, 'html5lib')
        stock = dict()

        sections = soup.find_all('g-card-section')

        # 第 2 個 g-card-section, 取出公司名及即時股價資訊
        stock['股票名稱'] = sections[1].div.text
        spans = sections[1].find_all('div')[3].find_all('span')
        stock['目前價格'] = soup.find = spans[0].text
        
        spans=spans[4].find_all('span')
        stock['價格改變'] = spans[0].text
        stock['價格改變%'] = spans[1].text

        # 第 4 個 g-card-section, 有左右兩個 table 分別存放股票資訊
        for table in sections[3].find_all('table'):
            for tr in table.find_all('tr')[:3]:
                key = tr.find_all('td')[0].text.lower().strip()
                value = tr.find_all('td')[1].text.strip()
                stock[key] = value
                
        stock=pd.DataFrame(stock.items(), columns=['item', 'value'])
        print(stock)


      item       value
0     股票名稱    台灣積體電路製造
1     目前價格  589.00 TWD
2     價格改變     −25.00 
3    價格改變%     (4.07%)
4       開盤      591.00
5       最高      595.00
6       最低      588.00
7       市值      15.27兆
8      本益比       28.25
9      殖利率       1.74%
10   上次收盤價      614.00
11  52 週高點      679.00
12  52 週低點      353.50


In [3]:
Y_FINANCE_HIS_URL = 'https://hk.finance.yahoo.com/quote/'
FINNANCE_NAME = '2330'
Y_URL=Y_FINANCE_HIS_URL+FINNANCE_NAME+'.TW/history?p='+FINNANCE_NAME+'.TW&.tsrc=fin-srch'

if __name__ == '__main__':
    #get_page
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                             'AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/66.0.3359.181 Safari/537.36'}
    resp = requests.get(Y_URL, headers=headers)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
    else:
        page=resp.text
    #get_information
    if page:
        soup = BeautifulSoup(page, 'html5lib')
        rows = soup.find('table').tbody.find_all('tr')
        stock = dict()
        stock=pd.DataFrame(stock.items(), columns=['日期', '開市','最高','最低','收市','經調整收市價','成交量'])
        
        
        
        for row in rows:
            try:
                x1=row.find_all('span')[0].text
                x2=row.find_all('span')[1].text
                x3=row.find_all('span')[2].text
                x4=row.find_all('span')[3].text
                x5=row.find_all('span')[4].text
                x6=row.find_all('span')[5].text
                x7=row.find_all('span')[6].text
            except:
                x1=None
                x2=None
                x3=None
                x4=None
                x5=None
                x6=None
                x7=None
            new_row = {'日期':x1,'開市':x2,'最高':x3,'最低':x4,'收市':x5,'經調整收市價':x6,'成交量':x7}
            stock = stock.append(new_row, ignore_index=True)

        print(stock)

            日期      開市      最高      最低      收市  經調整收市價          成交量
0   2021年7月16日  591.00  595.00  588.00  589.00  589.00   59,724,583
1   2021年7月15日  613.00  614.00  608.00  614.00  614.00   22,012,834
2   2021年7月14日  613.00  615.00  608.00  613.00  613.00   38,418,875
3   2021年7月13日  600.00  608.00  599.00  607.00  607.00   52,540,315
4   2021年7月12日  595.00  597.00  590.00  593.00  593.00   31,304,547
..         ...     ...     ...     ...     ...     ...          ...
95  2021年3月02日  621.00  622.00  609.00  609.00  604.01   54,041,533
96  2021年2月26日  611.00  618.00  606.00  606.00  601.03  111,861,175
97  2021年2月25日  636.00  636.00  628.00  635.00  629.80   43,231,417
98  2021年2月24日  627.00  636.00  625.00  625.00  619.88   66,206,750
99  2021年2月23日  641.00  643.00  633.00  641.00  635.75   50,076,333

[100 rows x 7 columns]


In [4]:
stock

,日期,開市,最高,最低,收市,經調整收市價,成交量
0,2021年7月16日,591.00,595.00,588.00,589.00,589.00,"59,724,583"
1,2021年7月15日,613.00,614.00,608.00,614.00,614.00,"22,012,834"
2,2021年7月14日,613.00,615.00,608.00,613.00,613.00,"38,418,875"
3,2021年7月13日,600.00,608.00,599.00,607.00,607.00,"52,540,315"
4,2021年7月12日,595.00,597.00,590.00,593.00,593.00,"31,304,547"
...,...,...,...,...,...,...,...
95,2021年3月02日,621.00,622.00,609.00,609.00,604.01,"54,041,533"
96,2021年2月26日,611.00,618.00,606.00,606.00,601.03,"111,861,175"
97,2021年2月25日,636.00,636.00,628.00,635.00,629.80,"43,231,417"
98,2021年2月24日,627.00,636.00,625.00,625.00,619.88,"66,206,750"
